In [ ]:
import numpy as np, pandas as pd, os, pickle
from natsort import natsorted
from scipy.signal import find_peaks
from datetime import timedelta
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output


print('Header files imported......')

In [ ]:
def plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et):
    if st != None and et!=None:
        print('I came inside......')
        start_date = st #pd.to_datetime('2023-07-21 00:00:00')
        end_date = et #pd.to_datetime('2023-07-26 00:00:00')

    df_combined_non_temp_humidity = []
    df_combined_temp_humidity = {}
    for sens in range(len(environmental_sensors)):
        sensor_name = environmental_sensors[sens]
        
        print(sens, sensor_name)
        if sensor_name in ['Stove_Hum_Temp_humidity','Stove_Hum_Temp_temp']:
            df_sensor = data_subject_2[sensor_name][0]
            df_sensor['ts_datetime'] = pd.to_datetime(df_sensor['ts_datetime'])

            if st != None and et != None:
                filtered_df = df_sensor[(df_sensor['ts_datetime'] >= start_date) & (df_sensor['ts_datetime'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()

            sub_df_sensor = filtered_df[['sensor_id', 'ts_datetime', 'sensor_status']]
            print(sub_df_sensor.head(),'\n')
            df_combined_temp_humidity[sensor_name] = sub_df_sensor
            sub_df_sensor = pd.DataFrame()
        elif sensor_name in ['Shower_Hum_Temp_humidity','Shower_Hum_Temp_temp']:
            continue
        else:
            df_sensor = data_subject_2[sensor_name][1]
            df_sensor['ts_on'] = pd.to_datetime(df_sensor['ts_on'])
            df_sensor['ts_off'] = pd.to_datetime(df_sensor['ts_off'])
            if st != None and et!=None:
                filtered_df = df_sensor[(df_sensor['ts_on'] >= start_date) & (df_sensor['ts_on'] <= end_date)]
            else:
                filtered_df = df_sensor.copy()
            sub_df_sensor = filtered_df[['sensor_id', 'ts_on', 'ts_off']]
            df_combined_non_temp_humidity.append(sub_df_sensor)
            sub_df_sensor = pd.DataFrame()

    df_combined_non_temp_humidity = pd.concat(df_combined_non_temp_humidity, ignore_index=True)
    df_combined_non_temp_humidity = df_combined_non_temp_humidity.sort_values(by=['sensor_id'], ascending=True)

    # clear_output(wait=True)

    fig = make_subplots(
        rows=3, 
        cols=1, 
        shared_xaxes=True, 
        row_heights=[0.1, 0.1, 0.4],
        vertical_spacing = 0.01
    ) 

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_temp'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_temp'].reset_index().sensor_status, 
            mode='lines', 
            name='Temperature', 
            line=dict(color='red')
        ), 
        row=1, 
        col=1
    )

    fig.add_trace(
        go.Scatter(
            x=df_combined_temp_humidity['Stove_Hum_Temp_humidity'].reset_index().ts_datetime,
            y=df_combined_temp_humidity['Stove_Hum_Temp_humidity'].reset_index().sensor_status,  
            mode='lines', 
            name='Humidity', 
            line=dict(color='blue')
        ), 
        row=2, 
        col=1
    )
    fig1 = go.Figure(px.timeline(df_combined_non_temp_humidity, x_start='ts_on', x_end='ts_off', y='sensor_id', color='sensor_id',labels={'sensor_id': 'Sensor ID'}))
    for i in range(len(fig1.data)):
        fig.append_trace(fig1.data[i],row=3,col=1)


    fig.update_layout(barmode='overlay')
    fig.update_yaxes(fixedrange=True)
    fig.update_layout(height=500) 
    fig.update_xaxes(type='date')
    fig.show()        


In [ ]:
with open('/home/hubble/work/serenade/src/data_matteo_upto_september_25_2024.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
subjects = list(data.keys())
print(subjects)

In [ ]:

subject = subjects[1]
print(subject)
data_subject_2 = data[subject]
environmental_sensors = list(data_subject_2.keys())
st = pd.to_datetime('2024-09-01 00:00:00')
et = pd.to_datetime('2024-09-30 23:59:59')
plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et)

In [ ]:

subject = subjects[0]
print(subject)
data_subject_2 = data[subject]
environmental_sensors = list(data_subject_2.keys())
st = pd.to_datetime('2024-09-01 00:00:00')
et = pd.to_datetime('2024-09-30 23:59:59')
plot_sensor_active_periods(data_subject_2, environmental_sensors, st, et)

In [ ]:
import pandas as pd, pickle, numpy as np
from scipy.signal import lombscargle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

with open('/home/hubble/work/serenade/data/data_matteo_upto_september_25_2024_corrected.pkl', 'rb') as file:
    data = pickle.load(file)
    

In [ ]:

def plot_sensor_readings_v1(arranged_data_np, complete_days):
    # Create a heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=arranged_data_np,
        colorscale='Hot',  # You can change this to any other scale you prefer
        colorbar=dict(title='Sensor Status'),
    ))

    # Update layout to customize axes
    fig.update_layout(
        title='Sensor Readings as Image',
        xaxis_title='Time of the Day (HH:MM)',
        yaxis_title='Date',
        xaxis=dict(
            tickvals=np.arange(0, 1440, 60),  # Set tick values for the x-axis
            ticktext=[f'{h:02d}:00' for h in range(24)],  # Create tick labels in HH:MM format
        ),
        yaxis=dict(
            tickvals=np.arange(0, len(arranged_data_np), 10),  # Set tick values for the y-axis
            ticktext=[str(complete_days[i]) for i in np.arange(0, len(arranged_data_np), 5)],  # Date labels
        ),
        height=600,
        width=1000,
    )

    # Show the figure
    fig.show()



def arrange_data_by_day_numpy(df):
    df['ts_datetime'] = pd.to_datetime(df['ts_datetime'])
    df['sensor_status'] = pd.to_numeric(df['sensor_status'], errors='coerce')
    start_date = df['ts_datetime'].dt.date.min()
    end_date = df['ts_datetime'].dt.date.max()
    complete_days = pd.date_range(start=start_date, end=end_date, freq='D').date
    
    daily_data_list = []
    for day in complete_days:
        daily_data = df[df['ts_datetime'].dt.date == day]
        all_minutes = pd.date_range(start=pd.Timestamp(day), end=pd.Timestamp(day) + pd.Timedelta(days=1) - pd.Timedelta(minutes=1), freq='min')
        daily_resampled = daily_data.set_index('ts_datetime')['sensor_status'].resample('min').mean()
        daily_resampled = daily_resampled.reindex(all_minutes, fill_value=np.nan)
        daily_resampled = daily_resampled.fillna(0)  # Fill NaN with 0
        if daily_data.empty:
            daily_data_list.append(np.full((1440,), 0))  # 1440 minutes in a day filled with 255
        else:
            daily_data_list.append(daily_resampled.values)
    arranged_data_np = np.array(daily_data_list)
    plot_sensor_readings(arranged_data_np, complete_days)
    return arranged_data_np, complete_days

def arrange_data_by_day_numpy_environmentals(df):
    # Ensure timestamps are in datetime format
    df['ts_on'] = pd.to_datetime(df['ts_on'])
    df['ts_off'] = pd.to_datetime(df['ts_off'])
    
    # Create a date range for complete days
    start_date = df['ts_on'].dt.date.min()
    end_date = df['ts_on'].dt.date.max()
    complete_days = pd.date_range(start=start_date, end=end_date, freq='D').date
    
    daily_data_list = []
    for day in complete_days:
        # Filter data for the current day
        daily_data = df[df['ts_on'].dt.date == day]
        all_minutes = pd.date_range(start=pd.Timestamp(day), end=pd.Timestamp(day) + pd.Timedelta(days=1) - pd.Timedelta(minutes=1), freq='min')
        daily_resampled = np.zeros((1440,), dtype=int)  # 1440 minutes in a day
        
        if daily_data.empty:
            daily_data_list.append(daily_resampled) 
            continue  
        for _, row in daily_data.iterrows():
            start_minute = (row['ts_on'] - pd.Timestamp(day)).total_seconds() // 60
            end_minute = (row['ts_off'] - pd.Timestamp(day)).total_seconds() // 60
            
            if start_minute == end_minute:
                daily_resampled[int(start_minute)] = 1
            elif start_minute < end_minute:
                daily_resampled[int(start_minute):int(end_minute)] = 1  # Set the range to 1
        
        daily_data_list.append(daily_resampled)

    arranged_data_np = np.array(daily_data_list)
    plot_sensor_readings(arranged_data_np, complete_days)
    
    return arranged_data_np , complete_days


In [ ]:


def plot_sensor_readings(arranged_data_np, complete_days):
    # Create a heatmap using Plotly
    fig = go.Figure(data=go.Heatmap(
        z=arranged_data_np,
        colorscale='Hot',  # You can change this to any other scale you prefer
        colorbar=dict(title='Sensor Value/Status'),
        hovertemplate='<b>X:</b> %{x}<br>' +  # Time of the day
                      '<b>Y:</b> %{y}<br>' +  # Date
                      '<b>Sensor Value:</b> %{z}<br>' +  # Sensor value
                      '<extra></extra>',  # Remove the extra hover info
    ))

    # Update layout to customize axes
    fig.update_layout(
        title='',
        xaxis_title='Time of the Day (HH:MM)',
        yaxis_title='Date',
        xaxis=dict(
            tickvals=np.arange(0, 1440, 60),  # Set tick values for the x-axis
            ticktext=[f'{h:02d}:00' for h in range(24)],  # Create tick labels in HH:MM format
            tickangle=-90
        ),
        yaxis=dict(
            tickvals=np.arange(0, len(arranged_data_np), 10),  # Set tick values for the y-axis
            ticktext=[str(complete_days[i]) for i in np.arange(0, len(arranged_data_np), 10)],  # Date labels
        ),
        height=600,
        width=1000,
    )

    # Show the figure
    fig.show()



In [ ]:
subject_name = 'subject_3'
print(data[subject_name].keys())

In [ ]:
df = data[subject_name]['Stove_Hum_Temp_temp'][0].copy()
arranged_np_data = arrange_data_by_day_numpy(df)

In [ ]:
df = data[subject_name]['Stove_Hum_Temp_humidity'][0].copy()
arranged_np_data = arrange_data_by_day_numpy(df)

In [ ]:
df = data[subject_name]['Shower_Hum_Temp_temp'][0].copy()
arranged_np_data = arrange_data_by_day_numpy(df)

In [ ]:
df = data[subject_name]['Shower_Hum_Temp_humidity'][0].copy()
arranged_np_data = arrange_data_by_day_numpy(df)

In [ ]:
subject_name

In [ ]:
df = data[subject_name]['Refrigerator'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['Freezer'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['MotionKitchen'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:

df = data[subject_name]['FoodStorageKitchen'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['Dishes_Glasses'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['Microwave'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['Medicines'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['Printer'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['MotionBedroom'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['MotionPrimaryBathroom'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
# dring cloths

In [ ]:
df = data[subject_name]['MotionSecondaryBathroom'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
# dring cloths

In [ ]:
df = data['subject_3']['MotionLivingRoomSofa'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#dog

In [ ]:
df = data[s]['MotionLivingRoomTablet'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['HouseEntrance'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['FoodStorageLivingRoom'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#sliding door

In [ ]:
df = data[subject_name]['CoffeMachine'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)

In [ ]:
df = data[subject_name]['PlugTvHall'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#sliding door

In [ ]:
df = data[subject_name]['Medicines'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#sliding door

In [ ]:
df = data[subject_name]['MotionLivingRoomSofa'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#sliding door

In [ ]:
df = data[subject_name]['MotionLivingRoomTablet'][1].copy()
arranged_np_data = arrange_data_by_day_numpy_environmentals(df)
#sliding door

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data
v1 = np.random.rand(24)  # One value for each hour (0 to 23)
v2 = np.sin(np.linspace(0, 24, 1440))  # Continuous data (minute-wise, 1440 points)

# Create time axis
time_v1 = np.arange(24)  # Hourly time for v1
time_v2 = np.linspace(0, 24, len(v2))  # Continuous time for v2 (e.g., every minute)

# Plotting
plt.figure(figsize=(10, 6))

# Plot v1 (hourly data)
plt.plot(time_v1, v1, label='v1 (Hourly data)', marker='o', linestyle='-', color='blue')

